# COSC2968|COSC3053 - Foundations of Artificial Inteligence for STEM
# Assignment 3: Option A - Machine Learning
# Project: Weather Prediction Model for New York City
#### Team 2
- Nguyen Ngoc Dung (s)
- Le Dam Quan (s4031504)
- Nguyen Tran Ha Phan (s)
- Phan Tri Hung (s)
- Tran Quoc Hung (s)

#### Introduction
This notebook demonstrates the implementation of several Machine Learning models in a pipeline to generate forecasts for the future maximum temperature in New York City based on historical weather data. The dataset comprises the following characteristics: temperature, humidity, windspeed, cloud cover, and other variables associated to weather. Our approach involves data preprocessing, feature engineering, and application of diverse machine learning models. 